In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

In [2]:
crawled_liste = []

In [3]:
class Article:
    def __init__(self, title, lead, datum, author, text, url):
        self.title = title
        self.lead = lead
        self.datum = datum
        self.author = author
        self.text = text
        self.url = url

In [4]:
def inhalter():
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    title = soup.find(class_="ContentHead_text__NId_w")
    title = title.get_text()
    title = title.strip()

        
    try:
        lead = soup.find(class_="ContentHead_lead__1S033")
        lead = lead.get_text()
    except AttributeError:
        lead = ""
    
    try:
        author = soup.find(class_="ContentMetaInfo_author__3hPjj")
        author = author.get_text()
    except AttributeError:
        author = ""
        
    try:
        time_indy = soup.find(class_=re.compile("FullDateTime_root__35h8M ArticleContainer_content-width__pYdH3"))
        time_indy = time_indy.get_text()
 #       time_indy = time_indy.split(",")
 #       time = time_indy[1]
 #       time = time.get_text()
 #       date = time_indy[0]
 #       date = date.get_text()
    except AttributeError:
        time_indy = ""
 #       time = ""
 #       date = ""

        
        
    try:
        text = str()
        ts = soup.find_all(class_="HtmlText_root__2zbjl")
        for t in ts:
            t = t.get_text()
            t = t.replace("\n", " ")
            t = t.strip()
            text += t
    except AttributeError:
        text = ""
        
    url = page          
    
    crawled = Article(title, lead, time_indy, author, text, url)
    crawled_liste.append(crawled)
            
        

In [5]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

driver.get("https://www.tagesanzeiger.ch")

In [ ]:
#manueller Login via driver GUI
#p = pickle.dump(driver.get_cookies() , open("tagi_cookies.pkl","wb"))

In [6]:
cookies = pickle.load(open("tagi_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [7]:
with open("tagesanzeiger_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)

In [9]:
for page in ueb_pages:
    time.sleep(2)
    try:
        driver.get(page)
        inhalter()
    except TimeoutException:
        print("TimeOut")
        pass    

In [10]:
with open('tagi_art1.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter = csv.writer(csvfile, delimiter='|', quotechar='"')


    for article in crawled_liste:
        blogwriter.writerow( [article.title, article.lead, article.datum, article.author, article.text, article.url] )